# 📊 SentiBR - Análise Exploratória de Dados (EDA)

Este notebook realiza uma análise exploratória completa do dataset de reviews.

## Objetivos:
1. Entender a distribuição dos dados
2. Analisar padrões de sentimento
3. Identificar características textuais
4. Avaliar balanceamento das classes
5. Definir estratégia de preprocessamento

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import re
import emoji
from pathlib import Path

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

%matplotlib inline

## 1. Carregar Dados

In [ ]:
# Carregar dataset processado
df = pd.read_csv('../../data/processed/processed_reviews.csv')

print(f"📊 Dataset carregado: {len(df)} reviews")
print(f"📝 Colunas: {df.columns.tolist()}")
df.head()

In [ ]:
# Informações gerais
df.info()

## 2. Análise de Distribuição de Sentimentos

In [ ]:
# Distribuição de sentimentos
sentiment_counts = df['sentiment'].value_counts()

fig = px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.values,
    labels={'x': 'Sentimento', 'y': 'Número de Reviews'},
    title='Distribuição de Sentimentos',
    color=sentiment_counts.index,
    color_discrete_map={
        'positivo': '#2ecc71',
        'neutro': '#f39c12',
        'negativo': '#e74c3c'
    }
)
fig.update_layout(showlegend=False, height=400)
fig.show()

print("\n📊 Estatísticas:")
print(sentiment_counts)
print(f"\n📈 Percentuais:")
print(sentiment_counts / len(df) * 100)

In [ ]:
# Distribuição de ratings
if 'overall_rating' in df.columns:
    fig = px.histogram(
        df, 
        x='overall_rating',
        title='Distribuição de Ratings (1-5 estrelas)',
        labels={'overall_rating': 'Rating'},
        nbins=5,
        color='sentiment'
    )
    fig.show()
    
    print(f"⭐ Rating médio: {df['overall_rating'].mean():.2f}")
    print(f"⭐ Rating mediano: {df['overall_rating'].median():.2f}")

## 3. Análise Textual

In [ ]:
# Comprimento das reviews
df['text_length'] = df['review_text'].str.len()
df['word_count'] = df['review_text'].str.split().str.len()

# Estatísticas por sentimento
print("📏 Comprimento médio das reviews (caracteres):")
print(df.groupby('sentiment')['text_length'].describe())

print("\n📝 Número médio de palavras:")
print(df.groupby('sentiment')['word_count'].describe())

In [ ]:
# Visualização de comprimento por sentimento
fig = px.box(
    df,
    x='sentiment',
    y='word_count',
    title='Distribuição do Número de Palavras por Sentimento',
    labels={'word_count': 'Número de Palavras', 'sentiment': 'Sentimento'},
    color='sentiment'
)
fig.show()

## 4. WordClouds por Sentimento

In [ ]:
# WordClouds
from wordcloud import STOPWORDS

# Adicionar stopwords em português
stopwords_pt = set(STOPWORDS)
stopwords_pt.update([
    'produto', 'comprei', 'compra', 'chegou', 'veio', 'muito', 'bem',
    'pra', 'tá', 'ta', 'é', 'e', 'o', 'a', 'de', 'da', 'do', 'que',
    'em', 'um', 'uma', 'os', 'as', 'para', 'com', 'por', 'mais'
])

fig, axes = plt.subplots(1, 3, figsize=(20, 6))
sentiments = ['positivo', 'neutro', 'negativo']
colors = ['Greens', 'Oranges', 'Reds']

for idx, (sentiment, color) in enumerate(zip(sentiments, colors)):
    text = ' '.join(df[df['sentiment'] == sentiment]['review_text'].astype(str))
    
    wordcloud = WordCloud(
        width=600,
        height=400,
        background_color='white',
        stopwords=stopwords_pt,
        colormap=color,
        max_words=100
    ).generate(text)
    
    axes[idx].imshow(wordcloud, interpolation='bilinear')
    axes[idx].set_title(f'Reviews {sentiment.upper()}', fontsize=16, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 5. Análise de Aspectos

In [ ]:
# Analisar cobertura de aspectos
aspect_cols = [col for col in df.columns if col.startswith('has_') or col.startswith('aspect_')]

if aspect_cols:
    print("📋 Aspectos identificados nas reviews:")
    
    aspect_stats = {}
    for col in aspect_cols:
        aspect_name = col.replace('has_', '').replace('aspect_', '')
        if df[col].dtype == bool or df[col].dtype == 'bool':
            count = df[col].sum()
        else:
            count = df[col].notna().sum()
        
        aspect_stats[aspect_name] = count
    
    # Plotar
    fig = px.bar(
        x=list(aspect_stats.keys()),
        y=list(aspect_stats.values()),
        title='Frequência de Menção de Aspectos',
        labels={'x': 'Aspecto', 'y': 'Número de Menções'}
    )
    fig.show()
    
    # Tabela
    aspect_df = pd.DataFrame({
        'Aspecto': list(aspect_stats.keys()),
        'Menções': list(aspect_stats.values()),
        'Percentual': [v/len(df)*100 for v in aspect_stats.values()]
    })
    print(aspect_df)

## 6. Análise de Emojis e Caracteres Especiais

In [ ]:
# Função para extrair emojis
def extract_emojis(text):
    return [c for c in text if c in emoji.EMOJI_DATA]

# Contar emojis
df['has_emoji'] = df['review_text'].apply(lambda x: len(extract_emojis(str(x))) > 0)
df['emoji_count'] = df['review_text'].apply(lambda x: len(extract_emojis(str(x))))

print(f"😊 Reviews com emojis: {df['has_emoji'].sum()} ({df['has_emoji'].sum()/len(df)*100:.1f}%)")
print(f"😊 Total de emojis: {df['emoji_count'].sum()}")

# Emojis por sentimento
emoji_by_sentiment = df.groupby('sentiment')['emoji_count'].sum()
print("\n😊 Emojis por sentimento:")
print(emoji_by_sentiment)

## 7. Exemplos de Reviews

In [ ]:
# Mostrar exemplos de cada sentimento
print("\n" + "="*80)
print("📝 EXEMPLOS DE REVIEWS POR SENTIMENTO")
print("="*80)

for sentiment in ['positivo', 'neutro', 'negativo']:
    print(f"\n🎯 SENTIMENTO: {sentiment.upper()}")
    print("-" * 80)
    
    samples = df[df['sentiment'] == sentiment].sample(min(3, len(df[df['sentiment'] == sentiment])))
    
    for idx, row in samples.iterrows():
        print(f"\n📌 Review {idx+1}:")
        if 'review_title' in row and pd.notna(row['review_title']):
            print(f"   Título: {row['review_title']}")
        if 'overall_rating' in row and pd.notna(row['overall_rating']):
            print(f"   Rating: {'⭐' * int(row['overall_rating'])} ({row['overall_rating']})")
        print(f"   Texto: {row['review_text'][:200]}...")
        print()

## 8. Balanceamento de Classes

In [ ]:
# Análise de balanceamento
class_distribution = df['sentiment'].value_counts()
max_count = class_distribution.max()
min_count = class_distribution.min()
imbalance_ratio = max_count / min_count

print("⚖️ Análise de Balanceamento de Classes:")
print(f"\n   Classe majoritária: {class_distribution.idxmax()} ({max_count} reviews)")
print(f"   Classe minoritária: {class_distribution.idxmin()} ({min_count} reviews)")
print(f"   Razão de desbalanceamento: {imbalance_ratio:.2f}x")

if imbalance_ratio > 3:
    print("\n   ⚠️ ALERTA: Dataset significativamente desbalanceado!")
    print("   📌 Recomendações:")
    print("      - Considerar técnicas de balanceamento (SMOTE, undersampling, etc)")
    print("      - Usar class_weights no treinamento")
    print("      - Avaliar com métricas além de accuracy (F1-score, precision, recall)")
elif imbalance_ratio > 1.5:
    print("\n   ⚠️ Dataset moderadamente desbalanceado")
    print("   📌 Considerar usar class_weights no treinamento")
else:
    print("\n   ✅ Dataset razoavelmente balanceado")

## 9. Conclusões e Próximos Passos

In [ ]:
print("\n" + "="*80)
print("📊 RESUMO DA ANÁLISE EXPLORATÓRIA")
print("="*80)

print(f"\n1️⃣ DADOS GERAIS:")
print(f"   - Total de reviews: {len(df):,}")
print(f"   - Comprimento médio: {df['word_count'].mean():.0f} palavras")
print(f"   - Reviews com emojis: {df['has_emoji'].sum()/len(df)*100:.1f}%")

print(f"\n2️⃣ DISTRIBUIÇÃO DE SENTIMENTOS:")
for sentiment, count in class_distribution.items():
    print(f"   - {sentiment.capitalize()}: {count:,} ({count/len(df)*100:.1f}%)")

print(f"\n3️⃣ QUALIDADE DOS DADOS:")
print(f"   - Missing values: {df.isnull().sum().sum()}")
print(f"   - Duplicatas: {df.duplicated().sum()}")

print(f"\n4️⃣ PRÓXIMOS PASSOS:")
print("   ✅ Preprocessamento de texto (limpeza, normalização)")
print("   ✅ Tokenização com BERT tokenizer")
print("   ✅ Split train/val/test")
print("   ✅ Balanceamento de classes (se necessário)")
print("   ✅ Fine-tuning do modelo BERT")

print("\n" + "="*80)

In [ ]:
# Salvar estatísticas
stats = {
    'total_reviews': len(df),
    'sentiment_distribution': class_distribution.to_dict(),
    'avg_length': df['word_count'].mean(),
    'median_length': df['word_count'].median(),
    'has_emoji_pct': df['has_emoji'].sum() / len(df) * 100,
    'imbalance_ratio': imbalance_ratio
}

import json
with open('../../data/processed/dataset_statistics.json', 'w', encoding='utf-8') as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)

print("📁 Estatísticas salvas em: data/processed/dataset_statistics.json")